In [ ]:
import sys, numpy, pandas, polars, matplotlib, seaborn, sklearn
print(f"✅ Python {sys.version.split()[0]} | NumPy {numpy.__version__} | Pandas {pandas.__version__} | Polars {polars.__version__} | Matplotlib {matplotlib.__version__} | Seaborn {seaborn.__version__} | Scikit-learn {sklearn.__version__}")

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

ふざけたエラーに遭遇。
解決策...[自分用メモ：OpenAIのライブラリ使うと「TypeError: Client.__init__() got an unexpected keyword argument 'proxies'」が出る - Qiita](https://qiita.com/hatman621221/items/fafaf4ac0e906616e8bd)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは私はジョンといいます。"},
    ],
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-ByN3qPlS6TnWflRVxXv2Eoy5w2vxy",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。どんなことについてお話ししましょうか？",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1753728026,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_197a02a720",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 25,
    "total_tokens": 51,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}
